In [1]:

from __future__ import division
import numpy as np
import pyedflib
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:


f = pyedflib.EdfReader("ssdata/register/ADGU101504.rec")
print(f)

# Show what signals are in there
signal_labels = f.getSignalLabels()
print("signal_labels:", signal_labels)

# Get signal from channel 1 only
signal_1 = f.readSignal(1)
# Get sampling frequency
fs_1 = f.getSampleFrequency(1)

# Close reader
f._close()
del f

# Show numbers
print(fs_1) 


In [ ]:
marks_example = "ssdata/label/marks/FixedSS_ADGU101504.txt"
states_example = "ssdata/label/states/StagesOnly_ADGU101504.txt"

# Format is 1:SQ4  2:SQ3  3:SQ2  4:SQ1  5:REM  6:WA
states = np.loadtxt(states_example, dtype='i', delimiter=' ')
#print(states[0])

# First two are start and end for mark. Last two are validity and channel
marks_file = np.loadtxt(marks_example, dtype='i', delimiter=' ')
#print(marks_file[0, :])

#print(marks_file.shape)

#print(marks_file[marks_file[:, 5] == 1, :].shape)

#
path_marks_file = marks_example
marks_file = np.loadtxt(path_marks_file, dtype='i', delimiter=' ')
print(marks_file.shape)
marks_and_validity = marks_file[marks_file[:, 5] == 1][:, [0,1]]
print(marks_and_validity.shape)


In [ ]:

# Playground with pandas

a = np.array([[1,1,0.1],[1,2,0.05],[2,1,-0.1]])
print(a)
rows_list = []
for i in range(len(a)):
        dict1 = {}
        # get input row in dictionary format
        # key = col_name
        dict1.update({'ID_REG': a[i, 0]})
        dict1.update({'ID_SEG': a[i, 1]})
        dict1.update({'EEG_DATA': a[i, 2]})
        rows_list.append(dict1)
df = pd.DataFrame(rows_list, columns = rows_list[0].keys())  
print(df)


# Check binary
a = np.array([1,1,1,0,0,0,0,1,1,1,1,1,0,1])
np.array_equal(a, a.astype(bool))


In [1]:
import numpy as np
import pandas as pd
import pickle

train_path = "pickle_data/n2fft_dataframe_lite_train.pkl"
train_df = pd.read_pickle(train_path)

features = np.stack(train_df['FFT_DATA'].values)
labels = train_df['MARK'].values


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'model_data/spindle_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C05771AAC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2018-06-27-03:43:49


INFO:tensorflow:Saving dict for global step 100: accuracy = 0.8188402, accuracy_baseline = 0.8558505, auc = 0.71761984, auc_precision_recall = 0.38664848, average_loss = 0.44564444, global_step = 100, label/mean = 0.14414948, loss = 56.972008, prediction/mean = 0.2466226


{'accuracy': 0.8188402, 'accuracy_baseline': 0.8558505, 'auc': 0.71761984, 'auc_precision_recall': 0.38664848, 'average_loss': 0.44564444, 'label/mean': 0.14414948, 'loss': 56.972008, 'prediction/mean': 0.2466226, 'global_step': 100}


In [3]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd


def main():
    # Load dataset
    train_path = "pickle_data/n2fft_dataframe_lite_train.pkl"
    test_path = "pickle_data/n2fft_dataframe_lite_test.pkl"

    # Training set
    train_df = pd.read_pickle(train_path)
    train_features = np.stack(train_df['FFT_DATA'].values)
    train_labels = train_df['MARK'].values.astype(int)
    del train_df
    
    # Create weight for training
    train_weight = np.ones(train_features.shape[0])
    train_weight[train_labels == 1] = 4
    
    # Test set
    test_df = pd.read_pickle(test_path)
    test_features = np.stack(test_df['FFT_DATA'].values)
    test_labels = test_df['MARK'].values.astype(int)
    del test_df

    # Specify that all features have real-value data
    feature_columns = [tf.feature_column.numeric_column("x", shape=train_features.shape[1])]
    weight_column = tf.feature_column.numeric_column("weight")

    # Build 2 layer DNN with 128, 64, 1 units respectively
    classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[128],
                                            n_classes=2,
                                            model_dir="model_data/spindle_estimator",
                                            optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
                                            weight_column=weight_column)

    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_features, "weight": train_weight},
        y=train_labels,
        num_epochs=None,
        shuffle=True,
        batch_size=32
    )

    # Train model
    classifier.train(input_fn=train_input_fn, steps=100)

    # Define the test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": test_features, "weight": np.ones(test_features.shape[0])},
        y=test_labels,
        num_epochs=1,
        shuffle=False)

    # Evaluate
    metrics = classifier.evaluate(input_fn=test_input_fn)
    print(metrics)
    # print("\nTest Precision: {0:f}\n".format(metrics["precision"]))
    # print("\nTest Recall: {0:f}\n".format(metrics["recall"]))


if __name__ == "__main__":
    main()
    
    


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'model_data/spindle_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C05771AAC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2018-06-27-03:43:49


INFO:tensorflow:Saving dict for global step 100: accuracy = 0.8188402, accuracy_baseline = 0.8558505, auc = 0.71761984, auc_precision_recall = 0.38664848, average_loss = 0.44564444, global_step = 100, label/mean = 0.14414948, loss = 56.972008, prediction/mean = 0.2466226


{'accuracy': 0.8188402, 'accuracy_baseline': 0.8558505, 'auc': 0.71761984, 'auc_precision_recall': 0.38664848, 'average_loss': 0.44564444, 'label/mean': 0.14414948, 'loss': 56.972008, 'prediction/mean': 0.2466226, 'global_step': 100}


In [10]:

a = np.ones(10)
b = np.array([1,1,1,1,2,2,2,1,1,1])
a[b==2] = 10
print(a)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'model_data/spindle_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C05771AAC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2018-06-27-03:43:49


INFO:tensorflow:Saving dict for global step 100: accuracy = 0.8188402, accuracy_baseline = 0.8558505, auc = 0.71761984, auc_precision_recall = 0.38664848, average_loss = 0.44564444, global_step = 100, label/mean = 0.14414948, loss = 56.972008, prediction/mean = 0.2466226


{'accuracy': 0.8188402, 'accuracy_baseline': 0.8558505, 'auc': 0.71761984, 'auc_precision_recall': 0.38664848, 'average_loss': 0.44564444, 'label/mean': 0.14414948, 'loss': 56.972008, 'prediction/mean': 0.2466226, 'global_step': 100}
